In [1]:
%load_ext autoreload
%autoreload 2
import glob
import xarray as xr
from matplotlib import pyplot as plt
%matplotlib qt
import cartopy.crs as ccrs
import numpy as np
import sys
sys.path.append('/home/onno/Thesis/Scripts')
import my_tools



Normal 10 lead day error plot

In [ ]:
#list all the relevant files
filez = sorted(glob.glob('/media/onno/Algemeen/Thesis/GFS_mean/*envelope*CFS.nc'))
#create orthographic plot for each lead day
fig,axz = plt.subplots(2,5,subplot_kw={'projection': ccrs.Orthographic(0,90)})

#Looping over all lead day error files
for i, ax in enumerate(axz.flat):
    ds = xr.open_dataset(filez[i+1],decode_times=False).squeeze()
    #Discard 85 - 90 degrees north, because these data yield unrealistical large errors
    ds = ds.sel(lat=slice(85,0))
    #Create coastlines
    ax.coastlines()
    #Plotting the data
    im = ax.pcolormesh(ds.lon.values,ds.lat.values,ds.v.values,
                    transform=ccrs.PlateCarree(),
                    cmap='bwr',
                    vmin=-2,vmax=2)
    #Plotting the variable mean error below the graph
    ax.text(x=0.5,y=-0.15,s='Mean error = {:.2f}'.format(float(my_tools.weighted_average_area(ds))),
           transform=ax.transAxes,horizontalalignment='center',fontsize=12)
    #set gridlines
    gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True)
    ax.set_title('Day {}'.format(i+1))

#set figure size
fig.set_size_inches(20, 10)
fig.suptitle('GEFS mean envelope error compared to CFS')
#Make room for color bar and create color bar
fig.subplots_adjust(left=0.03,right=0.9)
cbax = fig.add_axes([0.925, 0.2, 0.025, 0.6])
fig.colorbar(im, cax=cbax)
cbax.set_ylabel('E (m/s)', rotation=90)
# fig.savefig('/media/onno/Algemeen/Thesis/GFS_mean/GEFS_forecast_mean_phasespeed_error.png')


Simple 1 lead day error plot


In [3]:
#list all files
filez = glob.glob('/media/onno/Algemeen/Thesis/GFS_mean/*envelope*CFS.nc')

#Loop over all files
for i, file in enumerate(filez):
    #open associated file for lead day
    ds = xr.open_dataset(file,decode_times=False).squeeze()
    ds = ds.sel(lat=slice(80,20))
    #create figure
    fig,ax = plt.subplots(subplot_kw={'projection': ccrs.Orthographic(0,90)})
    ax.coastlines()
    #plot data
    im = ax.pcolormesh(ds.lon.values,ds.lat.values,ds.v.values,
                    transform=ccrs.PlateCarree(),
                    cmap='bwr',
                    vmin=-2,vmax=2)
    #create colorbar
    cbar = fig.colorbar(im)
    cbar.set_label('E (m/s)', rotation=90)
    fig.suptitle('GEFS mean RWP Envelope error day {}'.format(i))
#     fig.savefig('/media/onno/Algemeen/Thesis/GFS_mean/GEFS_forecast_envelope_error_day_{:02d}'.format(i))
#     plt.close(fig)

Normal 10 day plot for each season

In [ ]:
#Loop over all seasons
seasons = ['MAM','JJA','SON','DJF']
for season in seasons:
    #list all files for each season
    filez = sorted(glob.glob('/media/onno/Algemeen/Thesis/GFS_mean_season/*phasespeed_{}*.nc'.format(season)))
    #create orthographic plot for each lead day
    fig,axz = plt.subplots(2,5,subplot_kw={'projection': ccrs.Orthographic(0,90)})
    fig.set_size_inches(20, 10)
    #Plot each mean error for each lead day
    for i, ax in enumerate(axz.flat):
        #open accompanying netcdf file for lead day error
        ds = xr.open_dataset(filez[i+1],decode_times=False).squeeze()
        ds = ds.sel(lat=slice(85,0))
        ax.coastlines()
        #plot data
        im = ax.pcolormesh(ds.lon.values,ds.lat.values,ds.v.values,
                    transform=ccrs.PlateCarree(),
                    cmap='bwr',
                    vmin=-2,vmax=2)
        #plot text with mean error belowhemisphere plot
        ax.text(x=0.5,y=-0.15,s='Mean error = {:.2f}'.format(float(my_tools.weighted_average_area(ds))),
           transform=ax.transAxes,horizontalalignment='center',fontsize=12)
        gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True)
        ax.set_title('Day {}'.format(i+1))

    # cbar = fig.colorbar(im)
    # cbar.set_label('E (m/s)', rotation=90)
    fig.suptitle('GEFS mean RWP phase speed error {} compared to ERA5'.format(season))
    #Set colorbar and make room for it
    fig.subplots_adjust(right=0.85)
    cbax = fig.add_axes([0.875, 0.2, 0.025, 0.6])
    fig.colorbar(im, cax=cbax)
    cbax.set_ylabel('Cp (m/s)', rotation=90)
    #Save figure and make sure it does not open a window
    plt.close(fig)
    fig.savefig('/media/onno/Algemeen/Thesis/GFS_mean_season/GEFS_forecast_phasespeed_error_{}.png'.format(season,i),dpi=300)